In [7]:
pip webdriver_manager.chrome

ERROR: unknown command "webdriver_manager.chrome"
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import dash
import openpyxl
import os
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

ModuleNotFoundError: No module named 'webdriver_manager'

In [15]:
#Instalando o driver
servico = Service(ChromeDriverManager().install())

#Abrindo o navegador
navegador = webdriver.Chrome(service=servico)

##Iniciando o processo dados
navegador.get('https://statusinvest.com.br/acoes/busca-avancada/')
time.sleep(2)
navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[3]/div/div/div/button[2]').click()
time.sleep(2)
navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[4]/div/div[1]/div[2]/a/span').click()
time.sleep(2)

#Fecha o navegador
navegador.quit()

## Renomeia o arquivo e move para pasta desejada
source = '/Users/scheiner/Downloads/statusinvest-busca-avancada.csv'

os.remove ('/Users/scheiner/Documents/GitHub/StatusInvest/stocks.csv')

dest = '/Users/scheiner/Documents/GitHub/StatusInvest/stocks.csv'
os.rename(source, dest) 

print("Source path renamed to destination path successfully.")

Source path renamed to destination path successfully.


In [36]:
### Tratamento dos dados importados:

df = pd.read_csv("stocks.csv", sep=';', decimal=',', thousands='.', encoding="utf-8")
    
df['P/L_IDEAL'] = df[' VPA'] / df[' LPA']
df['UP/DOWN'] = (df[' VPA'] / df['PRECO'])-1
    
valuation = df.loc[:, ['TICKER', 'PRECO', ' VPA' , 'DY', ' LPA', 'P/L', 'P/L_IDEAL', 'UP/DOWN', 'ROE', 
                           'P/VP', 'EV/EBIT' ,'LIQ. CORRENTE', 'MARG. LIQUIDA', 'DIV. LIQ. / PATRI.']]

valuation.fillna(value=0, inplace=True)
valuation.rename(columns={'LIQ. CORRENTE': "LIQ.CORR" }, inplace=True)
valuation.rename(columns={'MARG. LIQUIDA': "MARG.LIQ" }, inplace=True)
valuation.rename(columns={'DIV. LIQ. / PATRI.': "DIV.LIQ/PL" }, inplace=True)
    
    
valuation = valuation.loc[valuation['DY'] >= 1].loc[
    valuation['P/L'] >= 0.01].loc[valuation['P/L'] < 15].loc[
    valuation['ROE'] >= 6].loc[
    valuation['EV/EBIT'] >= 0.01].loc[valuation['EV/EBIT'] <= 15].loc[
    valuation['LIQ.CORR'] >= 0.5].loc[
    valuation['UP/DOWN'] >= 0.1]    

valuation.shape

(48, 14)

In [37]:
#Instalando o driver
servico = Service(ChromeDriverManager().install())

#Abrindo o navegador
navegador = webdriver.Chrome(service=servico)

ticker = valuation['TICKER']
lista_stocks = ticker.to_list()

lista_indicadores_stocks = []
for stocks in lista_stocks:

    #Criando url
    url = f"https://statusinvest.com.br/acoes/{stocks}"

    
    try:
        #Abrindo o navegador
        navegador.get(url)

        time.sleep(0.25)
    
        #Coletando infos
        PRECO = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[1]/div/div[1]/strong').text
        MIN_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[2]/div/div[2]/div/span[2]').text
        MAX_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[3]/div/div[2]/div/span[2]').text
        MIN_52S = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[2]/div/div[1]/strong').text
        MAX_52S = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[3]/div/div[1]/strong').text
        DY = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[4]/div/div[1]/strong').text
        GAIN_12M = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[5]/div/div[1]/strong').text
        GAIN_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[5]/div/div[2]/div/span[2]/b').text
        SEGMENTO = navegador.find_element(By.XPATH, '//*[@id="company-section"]/div[1]/div/div[3]/div/div[3]/div/div/div/a/strong').text
        
    

        #Criando dict
        dicionario = {  "stocks": stocks,
                        "MIN_MES": MIN_MES, 
                        "MAX_MES": MAX_MES, 
                        "MIN_52S": MIN_52S, 
                        "MAX_52S": MAX_52S, 
                        "VAL_12M": GAIN_12M,
                        "VAL_MES": GAIN_MES,
                        "SEGMENTO": SEGMENTO
                        
                        }

        #Criando lista de dicts
        lista_indicadores_stocks.append(dicionario)
    except Exception as e:
        print(e)

#Fecha o navegador
navegador.quit()


In [45]:
### Preparando os dados e Dataframe

list_stocks = pd.DataFrame.from_dict(lista_indicadores_stocks)

list_stocks = list_stocks.replace('-', '')
list_stocks = list_stocks.replace('-%', '')
list_stocks['MIN_MES'] = (list_stocks['MIN_MES']).str.lstrip('R$ ')
list_stocks['MAX_MES'] = (list_stocks['MAX_MES']).str.lstrip('R$ ')
list_stocks['MIN_MES'] = (list_stocks['MIN_MES']).str.strip('- ')
list_stocks['MAX_MES'] = (list_stocks['MAX_MES']).str.strip('- ')
list_stocks['VAL_12M'] = (list_stocks['VAL_12M']).str.rstrip('%')
list_stocks['VAL_MES'] = (list_stocks['VAL_MES']).str.rstrip('%')
list_stocks.to_csv('list_stocks.csv')

list_stocks = pd.read_csv("list_stocks.csv", sep=',', decimal=',', thousands='.', encoding="utf-8" )

list_stocks = list_stocks.drop(['Unnamed: 0'],axis=1)
list_stocks.fillna(value=0, inplace=True)


valuation.rename(columns={'TICKER': "TICKERS" }, inplace=True)

list_stocks.rename(columns={'stocks': "TICKERS" }, inplace=True)

investing = pd.merge(valuation, list_stocks, on='TICKERS')

value_invest = investing[['TICKERS', 'PRECO', 'VAL_12M', 'UP/DOWN', 'MIN_52S', 'MAX_52S', ' VPA', 'DY', ' LPA', 
                          'P/L' ,'P/L_IDEAL',  'ROE', 'P/VP', 'EV/EBIT', 'LIQ.CORR', 'MARG.LIQ', 'DIV.LIQ/PL', 'SEGMENTO']]

### PEGANDO O INDICE IBOVESPA NA B3
def get_tickers(indice):
  url = 'http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice={}&idioma=pt-br'.format(indice.upper())
  return (pd.read_html(url, decimal=',', thousands='.', index_col='Código')[0][:-1].index).to_list()

t = get_tickers('IBOV')
ibov = pd.DataFrame(t)
ibov['IBOVESPA'] = 'IBOV'
ibov.rename(columns={0: "TICKERS" }, inplace=True)
ibov = ibov


### PEGANDO O INDICE SMALL CAPS NA B3
def get_tickers(indice):
  url = 'http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice={}&idioma=pt-br'.format(indice.upper())
  return (pd.read_html(url, decimal=',', thousands='.', index_col='Código')[0][:-1].index).to_list()

u = get_tickers('SMLL')

smll = pd.DataFrame(u)
smll['SMALLCAP'] = 'SMLL'
smll.rename(columns={0: "TICKERS" }, inplace=True)
smll = smll

### PEGANDO O INDICE SMALL IDIV NA B3
def get_tickers(indice):
  url = 'http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice={}&idioma=pt-br'.format(indice.upper())
  return (pd.read_html(url, decimal=',', thousands='.', index_col='Código')[0][:-1].index).to_list()

v = get_tickers('IDIV')

idiv = pd.DataFrame(v)
idiv['DIVIDENDOS'] = 'IDIV'
idiv.rename(columns={0: "TICKERS" }, inplace=True)
idiv = idiv

# value_invest = pd.merge(value_invest, ibov, on='TICKERS', how = 'left').fillna(0)
# value_invest = pd.merge(value_invest, smll, on='TICKERS', how = 'left').fillna(0)
# value_invest = pd.merge(value_invest, idiv, on='TICKERS', how = 'left').fillna(0)

# value_invest['IBOVESPA'] = (value_invest['IBOVESPA']).str.strip('0')
# value_invest['SMALLCAP'] = (value_invest['SMALLCAP']).str.strip('0')
# value_invest['DIVIDENDOS'] = (value_invest['DIVIDENDOS']).str.strip('0')

# value_invest.fillna('', inplace=True)

value_invest = value_invest.loc[value_invest['DY'] >= 6].loc[
    value_invest['VAL_12M'] >= 1].loc[
    value_invest['EV/EBIT'] >= 0.01].loc[value_invest['EV/EBIT'] <= 15].loc[
    value_invest['DIV.LIQ/PL'] >= 0].loc[
    value_invest['UP/DOWN'] >= 0.1]

value_invest

,TICKERS,PRECO,VAL_12M,UP/DOWN,MIN_52S,MAX_52S,VPA,DY,LPA,P/L,P/L_IDEAL,ROE,P/VP,EV/EBIT,LIQ.CORR,MARG.LIQ,DIV.LIQ/PL,SEGMENTO
2,BBAS3,47.05,24.37,0.206164,29.19,50.89,56.75,9.77,11.36,4.14,4.995599,20.01,0.83,3.17,12.76,12.35,0.00,Bancos
5,BGIP3,26.24,1.23,0.347180,21.06,33.22,35.35,8.55,3.65,7.20,9.684932,10.32,0.74,6.38,0.76,5.27,0.00,Bancos
6,BGIP4,23.00,40.50,0.536957,15.08,27.39,35.35,10.73,3.65,6.31,9.684932,10.32,0.65,6.38,0.76,5.27,0.00,Bancos
7,BRSR3,12.39,1.89,0.885391,9.58,14.71,23.36,7.60,2.01,6.16,11.621891,8.61,0.53,5.59,2.81,5.73,0.00,Bancos
9,BRSR6,11.98,5.55,0.949917,8.56,15.00,23.36,7.86,2.01,5.96,11.621891,8.61,0.51,5.59,2.81,5.73,0.00,Bancos
18,CSMG3,17.02,18.11,0.171563,12.87,20.81,19.94,9.81,2.85,5.98,6.996491,14.28,0.85,6.49,1.19,16.06,0.43,Água e Saneamento
23,EUCA4,12.37,70.62,0.866613,6.07,13.25,23.09,6.39,2.80,4.42,8.246429,12.13,0.54,5.17,1.61,9.82,0.26,Madeira
25,GGBR3,21.94,20.88,0.270283,18.15,25.88,27.87,15.00,5.44,4.03,5.123162,19.52,0.79,3.34,2.94,12.54,0.13,Siderurgia
26,GGBR4,23.77,5.97,0.172486,22.43,32.10,27.87,13.84,5.44,4.37,5.123162,19.52,0.85,3.34,2.94,12.54,0.13,Siderurgia
27,GOAU3,10.95,20.73,0.536986,9.07,13.04,16.83,10.59,3.10,3.54,5.429032,18.39,0.65,1.20,3.03,4.20,0.32,Siderurgia


In [50]:
value_invest['CARTEIRA'] = ''

value_invest = value_invest[['TICKERS', 'PRECO', 'VAL_12M', 'UP/DOWN', 'MIN_52S', 'MAX_52S', ' VPA', 'DY', ' LPA', 
                          'P/L' ,'P/L_IDEAL',  'ROE', 'P/VP', 'EV/EBIT', 'LIQ.CORR', 'MARG.LIQ', 'DIV.LIQ/PL', 'SEGMENTO', 'CARTEIRA']]

value_invest

,TICKERS,PRECO,VAL_12M,UP/DOWN,MIN_52S,MAX_52S,VPA,DY,LPA,P/L,P/L_IDEAL,ROE,P/VP,EV/EBIT,LIQ.CORR,MARG.LIQ,DIV.LIQ/PL,SEGMENTO,CARTEIRA
2,BBAS3,47.05,24.37,0.206164,29.19,50.89,56.75,9.77,11.36,4.14,4.995599,20.01,0.83,3.17,12.76,12.35,0.00,Bancos,
5,BGIP3,26.24,1.23,0.347180,21.06,33.22,35.35,8.55,3.65,7.20,9.684932,10.32,0.74,6.38,0.76,5.27,0.00,Bancos,
6,BGIP4,23.00,40.50,0.536957,15.08,27.39,35.35,10.73,3.65,6.31,9.684932,10.32,0.65,6.38,0.76,5.27,0.00,Bancos,
7,BRSR3,12.39,1.89,0.885391,9.58,14.71,23.36,7.60,2.01,6.16,11.621891,8.61,0.53,5.59,2.81,5.73,0.00,Bancos,
9,BRSR6,11.98,5.55,0.949917,8.56,15.00,23.36,7.86,2.01,5.96,11.621891,8.61,0.51,5.59,2.81,5.73,0.00,Bancos,
18,CSMG3,17.02,18.11,0.171563,12.87,20.81,19.94,9.81,2.85,5.98,6.996491,14.28,0.85,6.49,1.19,16.06,0.43,Água e Saneamento,
23,EUCA4,12.37,70.62,0.866613,6.07,13.25,23.09,6.39,2.80,4.42,8.246429,12.13,0.54,5.17,1.61,9.82,0.26,Madeira,
25,GGBR3,21.94,20.88,0.270283,18.15,25.88,27.87,15.00,5.44,4.03,5.123162,19.52,0.79,3.34,2.94,12.54,0.13,Siderurgia,
26,GGBR4,23.77,5.97,0.172486,22.43,32.10,27.87,13.84,5.44,4.37,5.123162,19.52,0.85,3.34,2.94,12.54,0.13,Siderurgia,
27,GOAU3,10.95,20.73,0.536986,9.07,13.04,16.83,10.59,3.10,3.54,5.429032,18.39,0.65,1.20,3.03,4.20,0.32,Siderurgia,


In [ ]:
value_invest = value_invest.loc[value_invest['TICKERS'] == 'BBAS3'].append(value_invest.loc[value_invest['CARTEIRA': 'SIM']])